In [ ]:
import subprocess
import sys

def detup_sweep(gpu_num, config_name, sweep_id=None):
    if sweep_id is None:
        result = subprocess.run(['wandb', 'sweep', f'configs/{config_name}.yaml'], capture_output=True, text=True)
        sweep_id = str(result).split('/')[-1].split('\\')[0]
    script = f"""#!/bin/bash
    
    source ~/.bash_profile 2>/dev/null || source ~/.bashrc
    eval "$(conda shell.bash hook)"
    conda activate evodiff
    wandb agent --name {config_name} alanamin/d3pm/{sweep_id}"""
    script_path = f'scripts/{config_name}_{sweep_id}.sh'
    with open(script_path, 'w+') as f:
        f.write(script)
    subprocess.run(['chmod', '+x', script_path])
    print("Sweep id:", sweep_id, "\nCommand:")
    print(f"cd ~/d3pm; nohup bash -c 'CUDA_VISIBLE_DEVICES={gpu_num} ~/d3pm/{script_path}' > ~/d3pm/outputs/{config_name}_{sweep_id}.log 2>&1 &")

In [ ]:
gpu_num = 1
config_name = 'sweep_gamma_cifar_params_agent_2'
sweep_id = "x38oy66z"
detup_sweep(gpu_num, config_name, sweep_id)

## slurm

In [ ]:
import numpy as np
import os
import subprocess
import wandb

def create_slurm_job(config_name, sweep_id=None, a100=False, n_gpus=1, n_cpus=32, run=False, time=48):
    # Create a new wandb sweep if sweep_id is not provided
    if sweep_id is None:
        result = subprocess.run(['wandb', 'sweep', '--name', config_name, f'configs/{config_name}.yaml'], capture_output=True, text=True)
        print(result)
        sweep_id = str(result).split('/')[-1].split('\\')[0]

    rand = np.random.randint(1000000)
    # Create a SLURM job script
    slurm_script = f"""#!/bin/bash
#SBATCH --job-name=sweeps{rand}
#SBATCH --output=slurm_out/slurm_{sweep_id}_{rand}_%j.out
#SBATCH --error=slurm_out/slurm_{sweep_id}_{rand}_%j.err
#SBATCH --time={time}:00:00
#SBATCH --ntasks=1
#SBATCH --mem=100GB
#SBATCH --cpus-per-task={n_cpus}
#SBATCH --gres=gpu:{'a100:' if a100 else ''}{n_gpus}


# Activate your virtual environment if needed
cd /scratch/aa11803/d3pm
source /scratch/aa11803/.bashrc
conda activate evodiff

# Run the wandb agent
wandb agent alanamin/d3pm/{sweep_id}
"""

    # Write the SLURM script to a file
    script_filename = f"/scratch/aa11803/d3pm/slurm_scripts/slurm_job_{sweep_id}_{rand}.sh"
    with open(script_filename, 'w') as f:
        f.write(slurm_script)

    # Submit the SLURM job
    print('sbatch ' + script_filename)
    if run:
        subprocess.run(['sbatch', script_filename])

    return sweep_id


In [ ]:
# config_name = 'sweep_process_cifar_small_sigma'
# sweep_id = None
# id_ = create_slurm_job(config_name, sweep_id)
# for i in range(1):
#     create_slurm_job(None, id_)
# print(config_name, id_)

# config_name = 'sweep_gamma_cifar'
# sweep_id = None
# id_ = create_slurm_job(config_name, sweep_id)
# for i in range(5):
#     create_slurm_job(None, id_)
# print(config_name, id_)

config_name = 'sweep_gamma_cifar_gaussian_0.05'
sweep_id = None
id_ = create_slurm_job(config_name, sweep_id)
for i in range(15):
    create_slurm_job(None, id_)
print(config_name, id_)

In [ ]:
config_name = 'full_scale_cifar_0.1'
sweep_id = None
create_slurm_job(config_name, sweep_id, a100=True, n_gpus=2, run=True)

In [ ]:
config_name = 'sweep_process_protein_pack_long'
sweep_id = None
create_slurm_job(config_name, sweep_id)

In [ ]:
config_name = 'full_scale_cifar'
create_slurm_job(config_name, None, a100=True, n_gpus=2, run=True)
config_name = 'full_scale_cifar_0.1'
create_slurm_job(config_name, None, a100=True, n_gpus=2, run=True)

In [ ]:
config_name = 'sweep_process_protein_pack_long'
sweep_id = None
id_ = create_slurm_job(config_name, sweep_id, run=True)
for i in range(1):
    create_slurm_job(None, id_, a100=True, run=True)
print(config_name, id_)


config_name = 'sweep_process_protein_pack_dit_long'
sweep_id = None
id_ = create_slurm_job(config_name, sweep_id, run=True)
for i in range(1):
    create_slurm_job(None, id_, a100=True, run=True)
print(config_name, id_)

In [ ]:
config_name = 'sweep_process_protein_pack_long_unif'
id_ = create_slurm_job(config_name, sweep_id, run=True, a100=True)
print(config_name, id_)


config_name = 'sweep_process_protein_pack_dit_long_unif'
id_ = create_slurm_job(config_name, sweep_id, run=True, a100=True)
print(config_name, id_)

In [ ]:
import torch
import multiprocessing

multiprocessing.cpu_count()


In [ ]:
config_name = 'sweep_process_protein_pack'
sweep_id = None
id_ = create_slurm_job(config_name, sweep_id, run=True)
for i in range(8):
    create_slurm_job(None, id_, a100=True, run=True)
print(config_name, id_)


config_name = 'sweep_process_protein_pack_dit'
sweep_id = None
id_ = create_slurm_job(config_name, sweep_id, run=True)
for i in range(8):
    create_slurm_job(None, id_, a100=True, run=True)
print(config_name, id_)

In [ ]:
# config_name = 'sweep_process_lm1b'
# sweep_id = None
# id_ = create_slurm_job(config_name, sweep_id, run=True)
for i in range(10):
    create_slurm_job(None, 'g32mi62v', a100=True, run=True)
# print(config_name, id_)


# config_name = 'test_param_process_lm1b'
# sweep_id = None
# id_ = create_slurm_job(config_name, sweep_id, run=True)
for i in range(4):
    create_slurm_job(None, 'u5k82zlj', a100=True, run=True)
# print(config_name, id_)

## Make masking and sedd

In [ ]:
import yaml

def make_baseline_configs(config_name, include_md=True):
    # Load the existing configuration
    with open(f'configs/{config_name}.yaml', 'r') as file:
        config = yaml.safe_load(file)
    
    if 'model.gamma' in config['parameters']:
        del config['parameters']['model.gamma']
    
    config['parameters']['architecture.nn_params.time_embed_dim'] = {'value': 512}
    config['parameters']['model.model'] = {'values': include_md * ['MaskingDiffusion'] + ['SEDD']}
    
    # Save the modified configuration
    with open(f'configs/baselines/{config_name}.yaml', 'w') as file:
        yaml.dump(config, file, default_flow_style=False)
    return f'baselines/{config_name}'

In [ ]:
config_name = 'full_scale_cifar'
config_name = make_baseline_configs(config_name, include_md=False)
create_slurm_job(config_name, None, a100=True, n_gpus=1)

config_name = 'full_scale_cifar_0.1'
config_name = make_baseline_configs(config_name, include_md=False)
create_slurm_job(config_name, None, a100=True, n_gpus=1, run=True)

In [ ]:
# config_name = 'sweep_process_cifar_small_sigma'
# config_name = make_baseline_configs(config_name, include_md=False)
# create_slurm_job(config_name, None)
# config_name = 'sweep_gamma_cifar'
# config_name = make_baseline_configs(config_name, include_md=True)
# create_slurm_job(config_name, None)
# config_name = 'sweep_gamma_cifar_gaussian'
# config_name = make_baseline_configs(config_name, include_md=False)
# create_slurm_job(config_name, None)
config_name = 'sweep_gamma_cifar_gaussian_0.1'
config_name = make_baseline_configs(config_name, include_md=False)
create_slurm_job(config_name, None)

In [ ]:
for i in range(3):
    create_slurm_job(None, 'ky01q6ds', run=False)